In [1]:
import glob
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import unicodedata
import re

In [2]:
class Flattening():
    '''
    f_type: flatening type
        + type_1: ă, â, ê, ...
        + type_2: type_1 + {đ, ơ, ư}
    '''
    def __init__(self, f_type='type_1'):
        self.f_type = f_type
        self.accent2unicode = {'<6>': '\u0302', '<7>': '\u031B', '<8>': '\u0306', '<F>': '\u0300', \
                               '<S>': '\u0301', '<R>': '\u0309', '<X>': '\u0303', '<J>': '\u0323'}
        self.circumflex_unicodes = ['00C2', '00E2', '00CA', '00EA', '00D4', '00F4'] # â, Â, Ê, ...
        self.breve_unicodes = ['0102', '0103'] # ă, Ă
        self.underdot_unicodes = ['1EA0', '1EA1', '1EB8', '1EB9', '1ECC', '1ECD']
        self._7_unicodes = ['01A0', '01A1', '01AF', '01B0']
        self.accent_letters = 'À Á Ả Ã Ạ Â Ầ Ấ Ẩ Ẫ Ậ Ă Ằ Ắ Ẳ Ẵ Ặ à á ả ã ạ â ầ ấ ẩ ẫ ậ ă ằ ắ ẳ ẵ ặ\
        È É Ẻ Ẽ Ẹ Ê Ề Ế Ể Ễ Ệ è é ẻ ẽ ẹ ê ề ế ể ễ ệ\
        Ì Í Ỉ Ĩ Ị ì í ỉ ĩ ị\
        Ò Ó Ỏ Õ Ọ Ô Ồ Ố Ổ Ỗ Ộ Ơ Ờ Ớ Ở Ỡ Ợ ò ó ỏ õ ọ ô ồ ố ổ ỗ ộ ơ ờ ớ ở ỡ ợ\
        Ù Ú Ủ Ũ Ụ Ư Ừ Ứ Ử Ữ Ự ù ú ủ ũ ụ ư ừ ứ ử ữ ự\
        Ỳ Ý Ỷ Ỹ Ỵ ỳ ý ỷ ỹ ỵ'
        if self.f_type=='type_2':
            self.accent_letters += ' đ Đ'
        self.accent_letters = self.accent_letters.split()
    
    def get_unaccent(self, letter):
        letter = letter.encode('utf-8').decode('utf-8')
        if self.f_type == 'type_1':
            letter = re.sub(u'[àáảãạâầấẩẫậăằắẳẵặ]', 'a', letter)
            letter = re.sub(u'[ÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶ]', 'A', letter)
            letter = re.sub(u'[èéẹẻẽêềếệểễ]', 'e', letter)
            letter = re.sub(u'[ÈÉẸẺẼÊỀẾỆỂỄ]', 'E', letter)
            letter = re.sub(u'[òóọỏõôồốộổỗ]', 'o', letter)
            letter = re.sub(u'[ÒÓỌỎÕÔỒỐỘỔỖ]', 'O', letter)
            letter = re.sub(u'[ơờớợởỡ]', 'ơ', letter)
            letter = re.sub(u'[ƠỜỚỢỞỠ]', 'Ơ', letter)
            letter = re.sub(u'[ìíịỉĩ]', 'i', letter)
            letter = re.sub(u'[ÌÍỊỈĨ]', 'I', letter)
            letter = re.sub(u'[ùúụủũ]', 'u', letter)
            letter = re.sub(u'[ÙÚỤỦŨ]', 'U', letter)
            letter = re.sub(u'[ưừứựửữ]', 'ư', letter)
            letter = re.sub(u'[ƯỪỨỰỬỮ]', 'Ư', letter)
            letter = re.sub(u'[ỳýỵỷỹ]', 'y', letter)
            letter = re.sub(u'[ỲÝỴỶỸ]', 'Y', letter)
            return letter
        elif self.f_type == 'type_2':
            letter = re.sub(u'đ', 'd', letter)
            letter = re.sub(u'Đ', 'D', letter)
            return ''.join(c for c in unicodedata.normalize('NFD', letter)\
                           if unicodedata.category(c) != 'Mn')
        else:
            raise ValueError('Should be: type_1 or type_2')

    def get_accents(self, letter):
        mark_accent, vowel_accent = None, None
        bi_unicode = unicodedata.decomposition(letter).split()

        if letter=='đ' or letter=='Đ':
            mark_accent = '<9>'
        elif bi_unicode[1]=='0302' or (bi_unicode[0] in self.circumflex_unicodes) or letter=='ậ' or letter=='Ậ':
            mark_accent = '<6>' # VNI '<CIRCUMFLEX>'
        elif bi_unicode[1]=='0306' or (bi_unicode[0] in self.breve_unicodes) or letter=='ặ' or letter=='Ặ':
            mark_accent = '<8>' # '<BREVE>'
        elif (self.f_type=='type_2') and (bi_unicode[1]=='031B' or (bi_unicode[0] in self._7_unicodes)):
            mark_accent = '<7>'
            
        if letter=='đ' or letter=='Đ':
            vowel_accent = None
        elif bi_unicode[1]=='0300':
            vowel_accent = '<F>'
        elif bi_unicode[1]=='0301':
            vowel_accent = '<S>'
        elif bi_unicode[1]=='0303':
            vowel_accent = '<X>'
        elif bi_unicode[1]=='0309':
            vowel_accent = '<R>'
        elif bi_unicode[1]=='0323' or (bi_unicode[0] in self.underdot_unicodes):
            vowel_accent = '<J>'

        return mark_accent, vowel_accent

    def flatten_letter(self, letter):
        if letter not in self.accent_letters:
            return letter, None, None
        unaccent_letter = self.get_unaccent(letter)
        mark_accent, vowel_accent = self.get_accents(letter)
        return unaccent_letter, mark_accent, vowel_accent
    
    def flatten_word(self, word):
        flattened_word = ''
        for letter in word:
            unaccent_letter, mark_accent, vowel_accent = self.flatten_letter(letter)
            flattened_word = flattened_word + unaccent_letter + (mark_accent if mark_accent!= None else '')\
            + (vowel_accent if vowel_accent!= None else '')
        return flattened_word
    
    def invert(self, flattened_word):
        accent_word = ''
        letters = re.findall(r'[\w]|<.*?>', flattened_word)
        for letter in letters:
            if len(letter) == 1:
                accent_word = accent_word + letter
            else: # accent
                if letter == '<9>':
                    accent_word = accent_word[:-1] + ('đ' if accent_word=='d' else 'Đ')
                else:
                    accent_word = accent_word + self.accent2unicode[letter]
                accent_word = unicodedata.normalize('NFC', accent_word)
        return accent_word

# type_1

In [7]:
flattening = Flattening(f_type='type_1')

string1 = 'quý hóa hoàn khoáng gì gìn đoán đứng lặng HĐND ơn'
string2 = []
for word in string1.split():
    word = re.findall(r'\w+', word)[0]
    flattened_word = flattening.flatten_word(word)
    accent_word = flattening.invert(flattened_word)
    print(f'{word: <{10}} - {flattened_word: <{20}} - {accent_word: <{10}}')
    string2.append(accent_word)
string2 = ' '.join(string2)

print('\n')
print(string1)
print(string2)
string1==string2 # punctuation->false

quý - quy<S> - quý
hóa - ho<S>a - hóa
hoàn - hoa<F>n - hoàn
khoáng - khoa<S>ng - khoáng
gì - gi<F> - gì
gìn - gi<F>n - gìn
đoán - đoa<S>n - đoán
đứng - đư<S>ng - đứng
lặng - la<8><J>ng - lặng
HĐND - HĐND - HĐND
ơn - ơn - ơn


quý hóa hoàn khoáng gì gìn đoán đứng lặng HĐND ơn
quý hóa hoàn khoáng gì gìn đoán đứng lặng HĐND ơn


True

In [8]:
df = pd.read_csv('../data/VNOnDB/all_word.csv', sep='\t')
ground_truth = df.loc[:, 'label'].astype(str)
accent_words = []
max_len_flattened = 0
for word in ground_truth:
    flattened_word = flattening.flatten_word(word)
    if len(re.findall(r'[\w]|<.*?>', flattened_word))>max_len_flattened:
        max_len_flattened = len(re.findall(r'[\w]|<.*?>', flattened_word))
        print(flattened_word)
    accent_word = flattening.invert(flattened_word)
    accent_words.append(accent_word)
    if word!=accent_word:
        print(word, '-', accent_word)
sum(ground_truth==accent_words)==len(ground_truth)

Ba<R>n
cha<6><S>t
chie<6><S>n
nghie<6><J>p
Bangladesh
Environment


True

# type_2

In [14]:
flattening = Flattening(f_type='type_2')

# string1 = 'Chống dịch như chống giặc - Cả nước cùng chung tay'
string1 = 'quý hóa hoàn khoáng gì gìn đoán đứng lặng HĐND'
string2 = []
for word in string1.split():
    word = re.findall(r'\w+', word)[0]
    flattened_word = flattening.flatten_word(word)
    accent_word = flattening.invert(flattened_word)
    print(f'{word: <{10}} - {flattened_word: <{20}} - {accent_word: <{10}}')
    string2.append(accent_word)
string2 = ' '.join(string2)

print('\n')
print(string1)
print(string2)
string1==string2 # punctuation->false

quý        - quy<S>               - quý       
hóa        - ho<S>a               - hóa       
hoàn       - hoa<F>n              - hoàn      
khoáng     - khoa<S>ng            - khoáng    
gì         - gi<F>                - gì        
gìn        - gi<F>n               - gìn       
đoán       - d<9>oa<S>n           - đoán      
đứng       - d<9>u<7><S>ng        - đứng      
lặng       - la<8><J>ng           - lặng      
HĐND       - HD<9>ND              - HĐND      


quý hóa hoàn khoáng gì gìn đoán đứng lặng HĐND
quý hóa hoàn khoáng gì gìn đoán đứng lặng HĐND


True

In [15]:
df = pd.read_csv('../data/VNOnDB/all_word.csv', sep='\t')
ground_truth = df.loc[:, 'label'].astype(str)
accent_words = []
max_len_flattened = 0
for word in ground_truth:
    flattened_word = flattening.flatten_word(word)
    if len(re.findall(r'[\w]|<.*?>', flattened_word))>max_len_flattened:
        max_len_flattened = len(re.findall(r'[\w]|<.*?>', flattened_word))
        print(flattened_word)
    accent_word = flattening.invert(flattened_word)
    accent_words.append(accent_word)
    if word!=accent_word:
        print(word, '-', accent_word)
sum(ground_truth==accent_words)==len(ground_truth)

Ba<R>n
cha<6><S>t
d<9>uo<6><R>i
d<9>u<7>o<7><J>c
tru<7>o<7><F>ng
Bangladesh
Environment


True